In [ ]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.flux_surfaces.flux_surface_meshing as fsm
import jax_sbgeom.flux_surfaces.flux_surfaces_base as fsb
import jax_sbgeom.flux_surfaces.flux_surfaces_extended as fse
from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh, _vertices_to_pyvista_polyline
import pyvista as pv
from dataclasses import dataclass
import h5py
import jax_sbgeom
import matplotlib.pyplot as plt


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]
stell_i = 2
vmec_file = vmec_files[stell_i]
coil_file = coil_files[stell_i]

In [ ]:
def _get_extended_no_phi_flux_surfaces(vmec_file):    
    fs_jax    = jsb.flux_surfaces.FluxSurfaceNormalExtendedNoPhi.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.Flux_Surfaces_Normal_Extended_No_Phi(SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file))
    return fs_jax, fs_sbgeom


def _get_discrete_coils(coil_file):
    with h5py.File(coil_file, 'r') as f:
        coil_data = jnp.array(f['Dataset1'])

    return jsb.coils.CoilSet.from_list([jsb.coils.DiscreteCoil.from_positions(coil_data[i]) for i in range(coil_data.shape[0])])

fs_jax, fs_sbgeom = _get_extended_no_phi_flux_surfaces(vmec_file)  # just to compile
coilset_jax       = _get_discrete_coils(coil_file)

import jax_sbgeom.jax_utils.raytracing as RT

In [ ]:
cws_mesh = jax_sbgeom.coils.coil_winding_surface.create_optimized_coil_winding_surface(coilset_jax, 100, False, 100)

In [ ]:
toroidal_line_test = cws_mesh[0].reshape(100, -1, 3)

In [ ]:
from scipy.interpolate import BSpline

In [ ]:
import scipy
y = jnp.concatenate([toroidal_line_test[0, :, 0], toroidal_line_test[0, :1, 0]])
t = jnp.linspace(0, 1, y.shape[0])

In [ ]:
y = jnp.concatenate([toroidal_line_test[:, :, :], toroidal_line_test[:, :1, :]], axis=1)
batched_y = jnp.moveaxis(y, -1, 0)

In [ ]:
t = jnp.linalg.norm(y[:,:]-jnp.roll(y,1,axis=1), axis=-1).cumsum(axis=1)
t = t / t[:,-1:]

In [ ]:
bspline_batch = jax_sbgeom.jax_utils.splines.periodic_interpolating_spline(t, batched_y, 3)

In [ ]:
polyline = bspline_batch(jnp.linspace(0, 1, 200))

In [ ]:
polysurf = jnp.moveaxis(polyline, 0, -1)

In [ ]:
connectivity = jax_sbgeom.flux_surfaces.flux_surface_meshing._mesh_surface_connectivity(polysurf.shape[0], polysurf.shape[1], True, True)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(jax_sbgeom.jax_utils.utils._vertices_to_pyvista_polyline(polysurf[50]), color='red', line_width=4)
plotter.add_mesh(jax_sbgeom.jax_utils.utils._vertices_to_pyvista_polyline(toroidal_line_test[0,:,:]), color='blue', line_width=4)
plotter.add_mesh(_mesh_to_pyvista_mesh(polysurf.reshape(-1,3), connectivity), opacity=1.0)
plotter.show()


In [ ]:
theta = jnp.linspace(0, 2 * jnp.pi, 210)
phi   = jnp.linspace(0, 2 * jnp.pi / fs_jax.settings.nfp, 100)
theta, phi = jnp.meshgrid(theta, phi, indexing='ij')
positions_lcfs_mg = fs_jax.cartesian_position(1.0,  theta, phi)
directions_lcfs_mg = fs_jax.cartesian_position(2.0, theta, phi) - positions_lcfs_mg

In [ ]:
@jax.jit
def find_minimum_distance_from_bvh(points, directions, mesh):
    bvh = RT.build_lbvh(mesh[0], mesh[1])
    hits_possible = RT.ray_traversal_bvh_vectorized(bvh, points, directions)    
    mesh_total = jnp.moveaxis(mesh[0][mesh[1][bvh.order[hits_possible]]], -3, 0)
    return jnp.nanmin(RT.ray_triangle_intersection_vectorized(points, directions, mesh_total), axis=0)

In [ ]:
import matplotlib.pyplot as plt
dmesh = find_minimum_distance_from_bvh(positions_lcfs_mg, directions_lcfs_mg, (polysurf.reshape(-1,3), connectivity)) 
plt.pcolormesh(phi, theta, dmesh)
plt.colorbar()

In [ ]:
from StellBlanket.SBGeom import Coils_jax as CJ

In [ ]:

def _get_all_fourier_coils(request):
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(coil_files[request])
    coilset_fourier = SBGeom.Coils.Convert_to_Fourier_Coils(coilset_sbgeom)
    
    coilset_jax = [jsb.coils.FourierCoil(jnp.array(i.Get_Fourier_Cos()), jnp.array(i.Get_Fourier_Sin()), jnp.array(i.Get_Centre())) for i in coilset_fourier]
    return coilset_jax, coilset_fourier

_, sbgeom_coilset = _get_all_fourier_coils(stell_i)

In [ ]:
splines = CJ.Create_Optimized_Splines(sbgeom_coilset, 100, 200)


In [ ]:
mesh_cws_sbgeom = SBGeom.Mesh.Mesh(splines.reshape(-1,3), SBGeom.Mesh.Connectivity_Toroidal_Surface_Mesh(200, 100, (0.0, 2 * onp.pi)))

In [ ]:
phi, theta, d_matrix = SBGeom.util.Generate_Thickness_Matrix_Raytracing(fs_sbgeom, mesh_cws_sbgeom, 100, 210)

In [ ]:
print(d_matrix, dmesh)

In [ ]:
plt.pcolormesh(phi, theta, d_matrix - dmesh, vmin =-0.2, vmax=0.2, cmap = 'bwr')
plt.colorbar()

In [ ]:
plotter = pv.Plotter()
#plotter.add_mesh(mesh_cws_sbgeom.to_pyvista())
plotter.add_mesh(_mesh_to_pyvista_mesh(*cws_mesh), color='red', opacity=1.0)
plotter.show()